In [4]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [5]:
SOC0,Cp,Np,Ns,K_charger = sym.symbols('SOC0,Cp,Np,Ns,K_charger',real=True)
Ibat,SOC,V1,V2 = sym.symbols('Ibat,SOC,V1,V2',real=True)


In [6]:
log = sym.log
exp = sym.exp

OCV_1=3.282-0.009294*log(SOC)-0.003919*log(1-SOC)-0.01518/SOC+0.06958*SOC; 
Rs_1=-0.6375*exp(-SOC) - 0.06625*SOC**3 + 0.2964*SOC**2 - 0.6335*SOC + 0.6393; 
R1_1=2.403*exp(-SOC) + 0.2176*SOC**3 - 1.059*SOC**2 + 2.356*SOC - 2.396; 
R2_1=2.29*exp(-SOC) + 0.1975*SOC**3 - 0.9956*SOC**2 + 2.239*SOC - 2.282;
C1_1=-1.405e7*SOC**4 + 2.873e7*SOC**3 - 1.919e7*SOC**2 + 4.442e6*SOC - 1.193e5;
C2_1=2.993e5*SOC**4 - 8.008e5*SOC**3 + 7.032e5*SOC**2 - 2.314e5*SOC + 3.088e4;

OCV_2=3.472 + 0.1094*log(SOC) - 0.03224*log(1 - SOC) + 0.001911/SOC - 0.2096*SOC;
Rs_2=0.5612*exp(-SOC) + 0.05642*SOC**3 - 0.2568*SOC**2 + 0.5546*SOC - 0.5588;
R1_2=6.319*exp(-SOC) + 0.6824*SOC**3 - 2.965*SOC**2 + 6.282*SOC - 6.316;
R2_2=4.688*exp(-SOC) + 0.5037*SOC**3 - 2.195*SOC**2 + 4.66*SOC - 4.686;
C1_2=-9.817e6*SOC**4 + 2.002e7*SOC**3 - 1.316e7*SOC**2 + 3.11e6*SOC - 1.451e5;
C2_2=1.864e5*SOC**4 - 3.158e5*SOC**3 + 1.497e5*SOC**2 - 1.201e4*SOC + 3615;
    

OCV = sym.Piecewise((OCV_1,Ibat>=0),(OCV_2,Ibat<0))
Rs  = sym.Piecewise(( Rs_1,Ibat>=0),( Rs_2,Ibat<0))
R1  = sym.Piecewise(( R1_1,Ibat>=0),( R1_2,Ibat<0))
R2  = sym.Piecewise(( R2_1,Ibat>=0),( R2_2,Ibat<0))
C1  = sym.Piecewise(( C1_1,Ibat>=0),( C1_2,Ibat<0))
C2  = sym.Piecewise(( C2_1,Ibat>=0),( C2_2,Ibat<0))

Icharger = K_charger*(SOC0 - SOC)
Iunid=(Ibat+Icharger)/Np;
dSOC=-(Ibat+Icharger)/(3600*Cp); 
dV1=-V1/(R1*C1)+Iunid/C1;
dV2=-V2/(R2*C2)+Iunid/C2;

Vunid=OCV-Rs*Iunid-V1-V2;
Vbat_calc=Vunid*Ns;

u_dict = {'Ibat':0.0, 'SOC0':0.5}
params_dict = {'Cp':66.2,'Np':2,'Ns':96,'K_charger':1e-6}
x_list = [  SOC,  V1,  V2]
f_list = [ dSOC, dV1, dV2]
y_list = [ ]
g_list = []
h_dict = {'Vbat_calc':Vbat_calc}

In [7]:
sys_dict = {'name':f"bess",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_dict,
       'u_ini_dict':u_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys_dict);
bldr.build()

system without algebraic equations, adding dummy algebraic equation


In [ ]:
function [dSOC,dV1,dV2,Vbat_calc] = fcn_batery(Ibat,SOC,V1,V2)
SOC0=1;
Cp=66.2;
Np=2;
Ns=96;

dSOC=-Ibat/(3600*Cp); 
if Ibat>=0 %las formulas de abajo estan expresadas para SOC de 0 a 1
    OCV=3.282-0.009294*log(SOC)-0.003919*log(1-SOC)-0.01518/SOC+0.06958*SOC; 
    Rs=-0.6375*exp(-SOC) - 0.06625*SOC^3 + 0.2964*SOC^2 - 0.6335*SOC + 0.6393; 
    R1=2.403*exp(-SOC) + 0.2176*SOC^3 - 1.059*SOC^2 + 2.356*SOC - 2.396; 
    R2=2.29*exp(-SOC) + 0.1975*SOC^3 - 0.9956*SOC^2 + 2.239*SOC - 2.282;
    C1=-1.405e7*SOC^4 + 2.873e7*SOC^3 - 1.919e7*SOC^2 + 4.442e6*SOC - 1.193e5;
    C2=2.993e5*SOC^4 - 8.008e5*SOC^3 + 7.032e5*SOC^2 - 2.314e5*SOC + 3.088e4;
else
    OCV=3.472 + 0.1094*log(SOC) - 0.03224*log(1 - SOC) + 0.001911/SOC - 0.2096*SOC;
    Rs=0.5612*exp(-SOC) + 0.05642*SOC^3 - 0.2568*SOC^2 + 0.5546*SOC - 0.5588;
    R1=6.319*exp(-SOC) + 0.6824*SOC^3 - 2.965*SOC^2 + 6.282*SOC - 6.316;
    R2=4.688*exp(-SOC) + 0.5037*SOC^3 - 2.195*SOC^2 + 4.66*SOC - 4.686;
    C1=-9.817e6*SOC^4 + 2.002e7*SOC^3 - 1.316e7*SOC^2 + 3.11e6*SOC - 1.451e5;
    C2=1.864e5*SOC^4 - 3.158e5*SOC^3 + 1.497e5*SOC^2 - 1.201e4*SOC + 3615;
end
Iunid=Ibat/Np;
dV1=-V1/(R1*C1)+Iunid/C1;
dV2=-V2/(R2*C2)+Iunid/C2;

Vunid=OCV-Rs*Iunid-V1-V2;
Vbat_calc=Vunid*Ns;
    
end
